In [1]:
import cv2

import ezkl
import os
import json

import tf2onnx
import onnx

import torch
from onnx2torch import convert

import sys
import numpy
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from collections import OrderedDict
from deepface import DeepFace

In [2]:
def ReadImage(pathname):
    img = DeepFace.extract_faces(img_path=pathname)[0]["face"]
    # img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    img = cv2.resize(img, (96, 96), interpolation=cv2.INTER_LINEAR)
    img = numpy.transpose(img, (2, 0, 1))
    print(img.shape)
    img = img.astype(numpy.float32) / 255.0
    # print(numpy.min(img), numpy.max(img))
    # print(numpy.sum(img[0]), numpy.sum(img[1]), numpy.sum(img[2]))
    I_ = torch.from_numpy(img).unsqueeze(0)

    return I_

img_paths = ['emmapassport.png']
imgs = []
for img_path in img_paths:
    imgs.append(ReadImage(img_path))

I_ = torch.cat(imgs, 0)
I_ = Variable(I_, requires_grad=False)

(3, 96, 96)


In [3]:
data_array = (I_.detach().numpy()).reshape([-1]).tolist()
data = dict(input_data = [data_array])
data_path = os.path.join('input.json')
settings_path = os.path.join('settings.json')
model_path = os.path.join('openface_pt_native.onnx')

In [4]:
# Serialize data into file:
json.dump( data, open(data_path, 'w' ))

py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "private"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "fixed" # private by default

# What is the settings file?
res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)

assert res == True

In [6]:
cal_path = os.path.join("calibration.json")

In [7]:

def ReadImage(pathname):
    img = DeepFace.extract_faces(img_path=pathname)[0]["face"]
    # img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    img = cv2.resize(img, (96, 96), interpolation=cv2.INTER_LINEAR)
    img = numpy.transpose(img, (2, 0, 1))
    print(img.shape)
    img = img.astype(numpy.float32) / 255.0
    # print(numpy.min(img), numpy.max(img))
    # print(numpy.sum(img[0]), numpy.sum(img[1]), numpy.sum(img[2]))
    I_ = torch.from_numpy(img).unsqueeze(0)

    return I_

img_paths = ['kayleepassport.jpg']
imgs = []
for img_path in img_paths:
    imgs.append(ReadImage(img_path))

I_ = torch.cat(imgs, 0)
I_ = Variable(I_, requires_grad=False)

data_array = (I_).reshape([-1]).tolist()
data = dict(input_data = [data_array])

json.dump(data, open(cal_path, 'w'))

res = await ezkl.calibrate_settings(cal_path, model_path, settings_path, target="resources", scales=[13], scale_rebase_multiplier=[1], only_range_check_rebase=True)

(3, 96, 96)




 <------------- Numerical Fidelity Report (input_scale: 13, param_scale: 13, scale_input_multiplier: 1) ------------->

+---------------+--------------+--------------+----------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error    | median_error | max_error    | min_error      | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+---------------+--------------+--------------+----------------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 0.00000773904 | 0            | 0.0003155172 | -0.00045087934 | 0.000045062527 | 0                | 0.00045087934 | 0             | 0.0000000055759783 | 0.0000144049045    | 0.00470821             |
+---------------+--------------+--------------+----------------+--------------

In [8]:
compiled_model_path = os.path.join('network.compiled')

In [9]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

In [7]:
res = await ezkl.get_srs( settings_path)

In [ ]:
# now generate the witness file 
witness_path = os.path.join('witness.json')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')

res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

In [ ]:
# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK



res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

PanicException: dynamic lookup or shuffle should only have one block

calibration failed max lookup input (-4202994348144, 118871353118596) is too large
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 5 columns for non-linearity table.
Using 9 columns for non-linearity table.
Using 9 columns for non-linearity table.
Using 9 columns for non-linearity table.
Using 9 columns for non-linearity table.
Using 9 columns for non-linearity table.
Using 9 columns for non-linearity table.
calibration failed extended k is too large to accommodate the quotient polynomial with logrows 20
circuit creation from run args failed: SigBitTruncationError
circuit creation from run args failed: SigBitTruncationError
calibration failed max lookup input (-78968271, 489000960) is too large
calibration failed max lookup input (-67320924713427, 1897707960036100) is too large
calibration failed max lookup input (-72087465, 